In [1]:
!pip install -U sentence-transformers 
!pip install --upgrade --no-cache-dir gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=1f535e8f93506aaae39d7bc7d1fa830d0d491255efcb35c669bdfe44836388f9
  Stored in directory: /root/.cache/pip/wheels/83/71/2b/40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence-transformers


In [2]:
!nvidia-smi 

Tue Apr  4 04:55:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [3]:
data_output_path = "data/"
teacher_model_url = "https://drive.google.com/uc?id=1P_NrfYL6XxVO95arSfomcgKMkvLxd-mX"
model_output_path = "model/"
data_url = "https://drive.google.com/uc?id=1k7mkiQvdNU7fpns4-0KZPQ4gMWa-vj_s"

In [ ]:
import gdown
gdown.download(teacher_model_url,model_output_path, quiet = False)
gdown.download(data_url, data_output_path,quiet=False)


In [5]:
!unzip /kaggle/working/model/mpnet_mnr_squad2.zip -d /kaggle/working/model

Archive:  /kaggle/working/model/mpnet_mnr_squad2.zip
  inflating: /kaggle/working/model/mpnet-mnr-squad2/1_Pooling/config.json  
  inflating: /kaggle/working/model/mpnet-mnr-squad2/README.md  
  inflating: /kaggle/working/model/mpnet-mnr-squad2/config.json  
  inflating: /kaggle/working/model/mpnet-mnr-squad2/config_sentence_transformers.json  
  inflating: /kaggle/working/model/mpnet-mnr-squad2/modules.json  
  inflating: /kaggle/working/model/mpnet-mnr-squad2/pytorch_model.bin  
  inflating: /kaggle/working/model/mpnet-mnr-squad2/sentence_bert_config.json  
  inflating: /kaggle/working/model/mpnet-mnr-squad2/special_tokens_map.json  
  inflating: /kaggle/working/model/mpnet-mnr-squad2/tokenizer.json  
  inflating: /kaggle/working/model/mpnet-mnr-squad2/tokenizer_config.json  
  inflating: /kaggle/working/model/mpnet-mnr-squad2/vocab.txt  


In [6]:
!cd /kaggle/working/model/mpnet-mnr-squad2/ && ls

1_Pooling			   sentence_bert_config.json
README.md			   special_tokens_map.json
config.json			   tokenizer.json
config_sentence_transformers.json  tokenizer_config.json
modules.json			   vocab.txt
pytorch_model.bin


In [7]:
teacher_model_location = '/kaggle/working/model/mpnet-mnr-squad2'

In [10]:
from sentence_transformers import SentenceTransformer, models

#Load the transformer model(Multi-Lingual)
transformer_model = models.Transformer('xlm-roberta-base')

# Add pooling layer
pooling_model = models.Pooling(
    transformer_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens= True
)

xlmr = models.Transformer('xlm-roberta-base')
pooler = models.Pooling(
    xlmr.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True
)




# # Create the sentence transformer model
student = SentenceTransformer(modules=[transformer_model, pooling_model])


student.to('cuda:1')
student


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [11]:
teacher = SentenceTransformer(teacher_model_location)
teacher.to('cuda:1')
teacher

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [12]:
batch_size = 32

from sentence_transformers import ParallelSentencesDataset

data = ParallelSentencesDataset(student_model=student, 
                                teacher_model=teacher,
                                batch_size=batch_size, 
                                use_embedding_cache=True)


### Data preparation 

In [ ]:
!tar -xf data/hasan-etal-2020-low.tar.bz2

In [ ]:
with open('hasan-etal-2020-low/2.75M/original_corpus.bn', 'r') as f:
    all_bangla_text = f.read()

In [ ]:
all_b_text = all_bangla_text.split("\n")

In [ ]:
with open('hasan-etal-2020-low/2.75M/original_corpus.en', 'r') as f:
    all_english_text = f.read()

In [ ]:
all_e_text = all_english_text.split("\n")

In [ ]:
e_text = all_e_text[1200000:1600000]
b_text = all_b_text[1200000:1600000]

In [ ]:
train_samples = []
for m , n in zip(e_text,b_text):
  train_samples.append(m+'\t'+n)

In [ ]:
import pickle

with open('e2b4lakh(12-16).bin','wb') as f:
  pickle.dump(train_samples,f)

In [ ]:
import gzip
import os

if not os.path.exists('./data'):
    os.mkdir('./data')

# save to file, sentence transformers reader will expect tsv.gz file
with gzip.open(f'./data/e2b4lakh(12-16).bin.tsv.gz', 'wt', encoding='utf-8') as f:
      f.write('\n'.join(train_samples))

In [ ]:
batch_size = 32

from sentence_transformers import ParallelSentencesDataset

data = ParallelSentencesDataset(student_model=student, 
                                teacher_model=teacher,
                                batch_size=batch_size, 
                                use_embedding_cache=True)

In [14]:
max_sentence_length = 512
data.load_data('data/e2b4lakh(12-16).bin.tsv.gz', max_sentence_length=max_sentence_length)

In [15]:
from torch.utils.data import DataLoader

loader = DataLoader(data, shuffle=True, batch_size=batch_size)

In [16]:
from sentence_transformers import losses

loss = losses.MSELoss(model=student)

In [17]:
out_dir = "TUNED_DPR_6LAKH_Epoch_14"
import os 
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

In [18]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [19]:
epochs = 2
warmup_steps = int(len(loader) * epochs * 0.1)

student.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    output_path=out_dir,
    optimizer_params={'lr': 2e-5, 'eps': 1e-6},#'correct_bias':False}, 
    save_best_model=True,
    show_progress_bar=True
)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/34408 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/sentence_transformers/SentenceTransformer.py:547: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:230.)
  labels = torch.tensor(labels)


Iteration:   0%|          | 0/34408 [00:00<?, ?it/s]